<a href="https://colab.research.google.com/github/Arunprabhakaran04/Meeting_assistant/blob/main/Meeting_Assistant_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Dependency

In [1]:
!pip -q install transformers
!pip -q install google-search-results
!pip -q install python-dotenv

  Preparing metadata (setup.py) ... done


In [2]:
!pip -q install langchain langchain-huggingface PyPDF2 langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.1 MB/s eta 0:00:00


In [31]:
!pip -q install faiss-cpu #customize it if we are using gpu.

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 27.5 MB/s eta 0:00:00


In [3]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from serpapi import GoogleSearch
import os

In [4]:
!pip -q install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.8 MB/s eta 0:00:00


In [5]:
!pip -q install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 kB 2.2 MB/s eta 0:00:00


In [6]:
import requests

In [7]:
import gradio as gr
import torch
import re

In [8]:
from groq import Groq

In [20]:
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [38]:
!pip -q install tavily-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00


In [39]:
from tavily import TavilyClient

##Groq-Whisper Model

In [9]:
from google.colab import userdata
groq_api_key = userdata.get("stt_bot_groq")

In [10]:
client = Groq(api_key=groq_api_key)

In [63]:
def transcribe_audio_with_groq(audio_path):
  if not audio_path:
    return ""
  with open(audio_path, "rb") as file:
      transcription = client.audio.transcriptions.create(
        file=(audio_path, file.read()),
        model="whisper-large-v3",
        language="en",
        temperature=0.1
      )

  return transcription.text


##Test1

In [13]:
audio_path = "/content/person1.m4a"
transcription = transcribe_audio_with_groq(audio_path)
print(transcription)

 Hi, this is Arun. I'm applying for the role of generated via internal company. Would you start the interview process for me?


##Question Extraction

In [14]:
def extract_question(transcription):
    classification_prompt = [
        {
            "role": "system",
            "content": (
              "You are a helpful assistant tasked with identifying the main question asked in a conversation."
              "Your goal is to extract the key question or query expressed by the user, even if it is implied or does not end with a question mark."
              "Focus on the user's intent, and prioritize their request for information, clarification, or guidance."
              "If no question is asked or the user's intent is unclear, return exactly: 'No clear question found.'"
              "Provide the output in a natural, human-readable sentence phrased as a clear question, and return only the question or the exact message: 'No clear question found.'"

            ),
        },
        {
            "role": "user",
            "content": f"Conversation:\n\"\"\"{transcription}\"\"\"."
        }
    ]

    try:
        response = client.chat.completions.create(
            messages=classification_prompt,
            model="llama-3.1-70b-versatile",
        )

        question_extracted = response.choices[0].message.content.strip()

        if question_extracted == "No clear question found.":
            return "No clear question found."

        return question_extracted

    except Exception as e:
        return f"Error in processing: {str(e)}"


##Text Classification

In [15]:
def classify_action(question):
    print("Classifying question using Zero-Shot Classification model...")

    classification_prompt = [
        {
            "role": "system",
            "content": (
                "You are an expert AI assistant trained to classify user queries into specific categories for a decision-making system.\n"
                "There are three categories to choose from:\n"
                "1. 'RAG' (Retrieval-Augmented Generation): Use this if the question requires retrieving detailed or structured knowledge from a database, document, or predefined dataset or about any project that the user has worked on.\n"
                "2. 'Web Search': Use this if the question is about real-time events, external facts, or knowledge not available in local datasets. Web search should only be selected for time-sensitive, factual, or real-world inquiries.\n"
                "3. 'LLM Answer': Use this if the question can be answered purely by a language model's general knowledge, reasoning capabilities, or linguistic understanding without external data.\n\n"
                "Guidelines:\n"
                "- Analyze the question's intent and content thoroughly before deciding.\n"
                "- If the query mentions current news, live updates, or external information, classify it as 'Web Search'.\n"
                "- If the query can be answered with general knowledge, logical reasoning, or basic facts, choose 'LLM Answer'.\n"
                "- If the query deals about personal project/interest or requires deep or structured knowledge not immediately available, classify it as 'RAG'.\n"
                "- Return only one category: RAG, Web Search, or LLM Answer.\n"
                "- Ensure the classification is precise and relevant."
            ),
        },
        {
            "role": "user",
            "content": f"User Query: {question}"
        }
    ]

    try:
        response = client.chat.completions.create(
            messages=classification_prompt,
            model="llama-3.1-70b-versatile",
        )

        classification = response.choices[0].message.content.strip()

        valid_labels = ["RAG", "Web Search", "LLM Answer"]
        if classification not in valid_labels:
            return "Error: Invalid classification returned."

        return classification

    except Exception as e:
        return f"Error in processing: {str(e)}"


##Test2

In [16]:
def sample_use_case():
    print("--- Sample Use Case: Classification Task ---")
    # Mocked transcription without a question mark, resembling voice output
    #mock_transcription = 'hi arun! hope you are doing well. this is in regard to your application for the role of gen ai developer. tell me more about your RAG project '
    mock_transcription = "hi XYZ! hope you are doing well. we received your application for the role of generative ai. we are happy with your projects and experience and we hope that you would be happy to be part of our company's ambition, what is the todays weather"
    print("Transcription:", mock_transcription)

    # Extract Question
    question = extract_question(mock_transcription)
    print("Extracted Question:", question)

    if question not in ["No clear question found.", "Error in processing"]:
        # Classify Action
        classification = classify_action(question)
        print("Action Decided:", classification)
    else:
        print("No question extracted or an error occurred.")

In [17]:
sample_use_case()

--- Sample Use Case: Classification Task ---
Transcription: hi XYZ! hope you are doing well. we received your application for the role of generative ai. we are happy with your projects and experience and we hope that you would be happy to be part of our company's ambition, what is the todays weather
Extracted Question: What is the today's weather?
Classifying question using Zero-Shot Classification model...
Action Decided: Web Search


##Building custom Agents

##Agent 1 : RAG

In [18]:
path = "/content/drive/MyDrive/Colab Notebooks/Meeting_Bot/Arun_Resume.pdf"

In [19]:
import os
from google.colab import userdata
sec_key = userdata.get("HF_TOKEN")
os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key


In [21]:
document_reader = PdfReader(path)

In [22]:
print(document_reader)

In [23]:
#reading data from the pages and adding it to a raw string
raw_text = ""
for i, pages in enumerate(document_reader.pages):
  text = pages.extract_text()
  if text:
    raw_text += text



In [24]:
len(raw_text)


3160

In [25]:
raw_text[:100]

'Arun\nPrabhakaran\nChennai,\nTamil\nNadu,\nIndia\n|\n+91-\n9841633771\n|\nLinkedIn\n|\nGitHub\n|\narunprabhakaran1'

In [26]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size= 1000,
    chunk_overlap = 200,
    length_function = len
)
texts = text_splitter.split_text(raw_text)

In [27]:
len(texts), type(texts)

(4, list)

In [28]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

<ipython-input-28-ad449aa8f6d2>:6: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/90.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [29]:
embeddings

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

In [32]:
doc_search = FAISS.from_texts(texts, embeddings)
doc_search.embedding_function

HuggingFaceBgeEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en', cache_folder=None, model_kwargs={'device': 'cpu'}, encode_kwargs={'normalize_embeddings': True}, query_instruction='Represent this question for searching relevant passages: ', embed_instruction='', show_progress=False)

##Test 3

In [ ]:
query = "acheivement in yantra"
docs = doc_search.similarity_search(query)
docs

[Document(id='12f54c66-bf79-4c92-bbf5-c126d6a2824e', metadata={}, page_content='ML\nclassification.\nCo-developing\nan\nAI-driven\nchatbo\nt\non\na\ncentralized\nweb\nplatform\n,\nenabling\nusers \nto\nquery\ncategorized\ndata\nand\nmake\ninformed\npurchasing\ndecisions.\nCurrently\nin\ndevelopment.\n●\nSJLT\nGroup\nMay’24\n-\nJuly’24 \nAI/ML\nSpecialist\n●\nDesigned\nand\ndeveloped\na\nmultilingual\nRAG\nchatbot\nfor\n100+\ndaily\nusers\n,\nenhancing\nefficiency\nby \naccurately\nretrieving\ninformation\nfrom\ncompany\nresources.\nBuilt\na\ndata\npipeline\nto\nstreamline\nupdates \nand\nadded\na\nvoice\nsearch\nfeature\nfor\nimproved\naccessibility.\nACHIEVEMENTS\n●\nSecured\n3rd\nplace\nin\nHelios\nat\nYantra\n2024\nTechnical\nFest\n.\nLed\na\nteam\nin\nideating\nsolutions\nfor\na\ndrone\nto \nexplore\na\ngas\ngiant.\nFocused\non\nsensor\nselection,\nconceptualizing\ndrone\nmodels,\nand\ndeveloping\nreinforcement \nlearning\nmodels\n,\nincluding\nDeep-Q\nNetworks\n,\nfor\nanalyzing\n

##LLM linking




In [33]:
from langchain_huggingface import HuggingFaceEndpoint
llm = HuggingFaceEndpoint(
    repo_id = "mistralai/Mistral-7B-Instruct-v0.3",
    temperature = 0.1,
    max_new_tokens = 512

)

In [34]:
from langchain.chains import RetrievalQA

retriever = doc_search.as_retriever(search_type="similarity", search_kwargs={"k":4})

rqa = RetrievalQA.from_chain_type(llm=llm,
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True)

##Rag function

In [35]:
def answer_query_rag(query):
  answer = rqa.invoke(query)["result"]
  return answer

##Agent 2 : LLM

In [64]:
def llm_answer_generation(question):
    prompt = [
        {
            "role": "system",
            "content": "You are a helpful assistant. Answer the user's question concisely and accurately."
        },
        {
            "role": "user",
            "content": question
        }
    ]

    try:
        response = client.chat.completions.create(
            messages=prompt,
            model="llama-3.1-70b-versatile",
        )
        answer = response.choices[0].message.content.strip()
        return answer

    except Exception as e:
        return f"Error in processing: {str(e)}"

##Agent 3 : websearch

In [40]:
from google.colab import userdata
tavily_api = userdata.get("Tavily_STT")
tavily_client = TavilyClient(api_key=tavily_api)

In [59]:
def web_search(query):
  print("Performing web search using Tavily API...")

  try:
      response = tavily_client.search(query)

      if response:
        ans = response["results"][0]["content"]
        confidence = response["results"][0]["score"]
        if confidence > 50:
          return ans;
        else:
          return llm_answer_generation(query)

      return llm_answer_generation(query)

  except Exception as e:
      return f"Error in processing: {str(e)}"

##Answer generation + Test 4

In [60]:
def get_final_answer(question, classification):
    if classification == "RAG":
        return answer_query_rag(question)
    elif classification == "Web Search":
        return web_search(question)
    elif classification == "LLM Answer":
        return llm_answer_generation(question)
    else:
        return "Unable to determine the appropriate action."

In [44]:
def sample_use_case():
    print("--- Sample Use Case: Full Workflow ---")

    mock_transcription = '''
    Hi there, Arun. Hope you are doing well. This is in regard to the internship position that you have applied at our company.
    explain more about your advanced image generation project.
    '''

    # mock_transcription = ''' hi there! how is your day going. this is in regard to the internship posistion that you have applied at our compnay.
    # who is the prime minister of india ?'''

    print("Transcription:", mock_transcription)
    question = extract_question(mock_transcription)
    print("Extracted Question:", question)

    classification = classify_action(question)
    print("Action Decided:", classification)

    answer = get_final_answer(question, classification)
    print("Final Answer:", answer)

In [45]:
sample_use_case()

--- Sample Use Case: Full Workflow ---
Transcription: 
    Hi there, Arun. Hope you are doing well. This is in regard to the internship position that you have applied at our company.
    explain more about your advanced image generation project.
    
Extracted Question: What can you tell us about your advanced image generation project?
Classifying question using Zero-Shot Classification model...
Action Decided: RAG
Final Answer:  The advanced image generation project involved implementing diverse deep learning methods, including Convolutional Neural Networks (CNNs), autoencoders, and Generative Adversarial Networks (GANs), to create lifelike medical images. Realistic images of brain tumors, chest X-rays, and polyps were generated to support medical research and diagnostics, thereby enhancing diagnostic precision and contributing to the advancement of medical research initiatives through synthetic medical imaging.


##Process Pipelining

In [47]:
overall_transcript = []
person_audio_paths = {"Person 1": [], "Person 2": []}

In [56]:
def process_pipeline(overall_transcription):
    if not overall_transcription.strip():
        return "No valid transcription provided.", "No question found.", "No classification.", "No answer."

    question = extract_question(overall_transcription)

    classification = classify_action(question)

    if classification == "RAG":
        answer = answer_query_rag(question)
    elif classification == "Web Search":
        answer = web_search(question)
    else:
        answer = llm_answer_generation(question)

    return overall_transcription, question, classification, answer

In [55]:
def record_audio(person, audio_file):
    global overall_transcript

    transcription = transcribe_audio_with_groq(audio_file)
    overall_transcript.append(f"{person}: {transcription}")

    full_transcript = "\n".join(overall_transcript)
    return full_transcript

In [51]:
def clear_transcripts():
    global overall_transcript, person_audio_paths
    overall_transcript = []
    person_audio_paths = {"Person 1": [], "Person 2": []}
    return "", "", "", ""

In [54]:
def gradio_interface():
    global overall_transcript

    full_transcription = "\n".join(overall_transcript)

    transcription, question, classification, answer = process_pipeline(full_transcription)

    return transcription, question, classification, answer

In [ ]:
ans = gradio_interface("/content/person1.m4a", "/content/person2.m4a")
print(ans[0])

Classifying question using Zero-Shot Classification model...
Generating answer using Llama model via Groq API...
Person 1: /content/person1.m4a
Person 2: /content/person1.m4a


##UI

In [61]:
with gr.Blocks() as demo:
    gr.Markdown("# Speech Query Processing System")

    with gr.Row():
        with gr.Column():
            person1_audio = gr.Audio(type="filepath", label="Person 1 Audio Input")
            person2_audio = gr.Audio(type="filepath", label="Person 2 Audio Input")
            person1_record_button = gr.Button("Record Person 1")
            person2_record_button = gr.Button("Record Person 2")
            submit_button = gr.Button("Help Me")
            clear_button = gr.Button("Clear")

        with gr.Column():
            overall_transcript_box = gr.Textbox(label="Overall Transcript", lines=10)
            question_box = gr.Textbox(label="Extracted Question")
            classification_box = gr.Textbox(label="Classification")
            answer_box = gr.Textbox(label="Final Answer")

    person1_record_button.click(
        fn=lambda audio: record_audio("Person 1", audio),
        inputs=person1_audio,
        outputs=overall_transcript_box
    )

    person2_record_button.click(
        fn=lambda audio: record_audio("Person 2", audio),
        inputs=person2_audio,
        outputs=overall_transcript_box
    )

    submit_button.click(
        fn=gradio_interface,
        inputs=[],
        outputs=[overall_transcript_box, question_box, classification_box, answer_box]
    )

    clear_button.click(
        fn=clear_transcripts,
        inputs=[],
        outputs=[overall_transcript_box, question_box, classification_box, answer_box]
    )

In [62]:
demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://4c658031be5169c54b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
